## <center>Plotly 3.0.0rc10<br><br>Mixed subplots including usually defined traces and at least one trace generated via figure_factory ##

This Jupyter notebook is intended for illustrating the new features of Plotly 3.0.0.

We are sampling from a  bivariate normal distributed data and define its scatter plot. On the other hand we extract the
X-marginal distribution from data points and generate its distplot. The corresponding figures are mixed into a subplot with 3 cells.

In [ ]:
from  plotly import figure_factory as FF
import plotly.graph_objs as go
from plotly.graph_objs import FigureWidget
import plotly.tools as tls
import numpy as np

Sample `n_pts` from the bivariate normal distribution of given mean vector and covariance matrix:

In [ ]:
n_pts=1000
m=[ 2.16, 3 ]#mean vector
Sigma=np.array([[1.2, 0.8],#covariance matrix
                [0.8, 1.75]])
x, y=np.random.multivariate_normal(m, Sigma, size=n_pts).T

In [ ]:
trace_biv_normal=go.Scatter(x=x,
                            y=y,
                            mode='markers',
                            marker=dict(size=6,
                                        color ='rgb(8, 104, 172)', # 'rgb(44, 127, 184)', #''rgb(37, 52, 148)', 
                                        line=dict(width=0.5, color='rgb(225,225,225)')),
                           )

Generate the distplot of the x-marginal distribution:

In [ ]:
f1 = go.FigureWidget(FF.create_distplot(hist_data=[x], group_labels=['distplot']))
f1

This figure consists in two vertically stacked subplots. In the upper cell is inserted the histogram and 
the probability density function, whereas in the lower one the rugplot:

In [ ]:
f1.data

We define a new  subplot of  two rows and two columns.  In the cell (1,2) we insert the 
histogram + the probability density function for the marginal distribution,  in the cell(2,2) the corresponding rugplot, and the cells (1,1) and (2, 1)  are concatenated to get one cell indexed  (1,1), where we insert the scatter plot (the bivariate distribution).

Such a structure is set via  specs (subplot specifications):

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
new_fig = tls.make_subplots(subplot_titles=('Scatter plot of the bivariate normal sampling', 'X-marginal distribution'),
                            specs=[[ {'rowspan': 2}, {}], #rowspan=2 means that we  concatenate 
                                                         #(1,1) with (1,2) into one cell encoded (1,1) , {}],
                                  [ None, {}]],                     
                            rows=2,
                            cols=2,
                            horizontal_spacing=0.075,
                            vertical_spacing=0.001
    )

In [ ]:
new_fig

In [ ]:
fws = go.FigureWidget(new_fig)

Now we add  the traces of the initially defined figures to the corresponding cell in the newly defined  subplot:

In [ ]:
fws.add_trace(trace_biv_normal, 1, 1)
fws.add_trace(f1.data[0], 1, 2)#Histogram
fws.add_trace(f1.data[1], 1, 2)#probability density
fws.add_trace(f1.data[2], 2, 2)#rugplot
fws

We transfer some of the `f1.layout.xaxis`, and `f1.layout.yaxis2` attributes to the corresponding axes in the `fws`, 
and  improve the general figure aesthetics using `fws.bach_update()`:

In [ ]:
f1.layout

In [ ]:
with fws.batch_update():
    fws.data[1].marker.line=dict(color='black', width=1.5)
    fws.data[1].autobinx=True#with  autobinx=False the distribution of data is poorly approximated,as one can see in f1
    fws.data[3].y=['rugplot']*n_pts#without this update it is displayed distplot, but I cannot understand why this string?
                                   #it isn't the yaxis3.title
    fws.layout.update(width=900, height=500, showlegend=False, plot_bgcolor='rgb(245,245,245)')
    fws.layout.yaxis1.update(zeroline=False, showline=True,  mirror=True, ticklen=4)
    fws.layout.xaxis1.update( zeroline=False, showline=True,  mirror=True, ticklen=4)
    fws.layout.yaxis2.ticklen=4
    fws.layout.xaxis2.ticklen=4
    fws.layout.xaxis3.update(zeroline= False, showticklabels=False, showgrid=False, ticklen=4)
fws